In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder
    .appName("WeatherDataProcessing")
    .master("local[*]")
    .config("spark.hadoop.io.nativeio.use.windows.nativeio", "false")
    .getOrCreate()
)


In [3]:
spark.range(5).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [4]:
input_path = "data/levi9-hack9-weather-firehose-2-2022-04-01-00-20-43-f2bad984-6327-36c6-997f-4ce326fd1df7"

In [5]:
df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(input_path)
)

In [6]:
df.show(5)
df.printSchema()

+--------------------+-------------------+-------------------+-----------------+------------------+--------------------+-------------------+-----------------+----------------+----------------+----------------+--------------+-----------------+---------------+
|                name|          time_nano|          time_date|location_latitude|location_longitude|       location_name|weather_temperature|weather_feelsLike|weather_pressure|weather_humidity|weather_dewPoint|weather_clouds|weather_windSpeed|weather_windDeg|
+--------------------+-------------------+-------------------+-----------------+------------------+--------------------+-------------------+-----------------+----------------+----------------+----------------+--------------+-----------------+---------------+
|1248 - Tătărași S...|1648771200000000000|2022-04-01 02:00:00|           47.154|            27.614|Tătărași Sud, Iaș...|              16.54|            15.61|             995|              52|            6.66|             0

In [7]:
import pyspark.sql.functions as F

In [8]:
df = df.withColumn(
    "datetime_iso",
    F.date_format(
        F.to_timestamp("time_date", "yyyy-MM-dd HH:mm:ss"),
        "yyyy-MM-dd'T'HH:mm:ss"
    )
)

In [9]:
df = df.withColumn(
    "wind_speed_kmh",
    F.col("weather_windSpeed").cast("double") * F.lit(3.6)
)

In [10]:
result = df.select(
    "name",
    "datetime_iso",
    F.col("weather_temperature").alias("temperature_c"),
    "wind_speed_kmh"
)

In [11]:
result.show(5, truncate=False)

+----------------------------------+-------------------+-------------+--------------+
|name                              |datetime_iso       |temperature_c|wind_speed_kmh|
+----------------------------------+-------------------+-------------+--------------+
|1248 - Tătărași Sud, Iași, Romania|2022-04-01T02:00:00|16.54        |11.268        |
+----------------------------------+-------------------+-------------+--------------+



In [13]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.4 MB 5.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.4 MB 5.1 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/11.4 MB 5.0 MB/s eta 0:00:02
   -------------- ------------------------- 4.2/11.4 MB 4.9 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.4 MB 4.9 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.4 MB 4.9 MB/s eta 0:00:02
   ------------------------ --------------- 7.1/11.4 MB 4.8 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.4 MB 4.8 MB/s eta 0:00:01
   -------------------------------- ------- 9.2/11.4 MB 4.9 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.4 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.4 MB 4.9 MB/s eta 0:00:01
   -------------

In [12]:
output_path = "output/weather_1248_2022-04-01"

(
    result
    .write
    .mode("overwrite")
    .option("header", True)
    .option("delimiter", ";")
    .csv(output_path)
)


Py4JJavaError: An error occurred while calling o57.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:288)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at

In [14]:
import os
os.makedirs("output", exist_ok=True)

pdf = result.toPandas()

output_file = "output/weather_1248_2022-04-01.csv"
pdf.to_csv(output_file, sep=";", index=False)

output_file

'output/weather_1248_2022-04-01.csv'